# Import libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

# Funções

In [2]:
def cria_modelos(df, df_pred, x_test, y_test):
    '''
    Função que cria modelos para a previsão
    '''
    X = df[x_test]
    y = df[y_test]

    model = GradientBoostingRegressor()
    model.fit(X,y)
    
    price_pred = model.predict(df_pred[x_test])
    
    return price_pred

# Read dataset

In [3]:
diamonds_test = pd.read_csv('data/diamonds.csv')
diamonds_rick = pd.read_csv('data/rick_diamonds.csv')
diamonds_test

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
1,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
2,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
3,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
4,0.24,Very Good,J,VVS2,62.8,57.0,336,3.94,3.96,2.48
...,...,...,...,...,...,...,...,...,...,...
48935,0.72,Premium,D,SI1,62.7,59.0,2757,5.69,5.73,3.58
48936,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
48937,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
48938,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56


# Data Cleaning 

In [4]:
# removendo as colunas de x,y,z que estiverem todas zeradas
diamonds_test = diamonds_test[~((diamonds_test['x'] == 0) & (diamonds_test['y'] == 0) & (diamonds_test['z'] == 0))]

In [5]:
# remoção das colunas com x =0, como todas nessas colunas y ou z também era =0
diamonds_test = diamonds_test[~(diamonds_test['x'] == 0)]

In [6]:
# não havia colunas y = 0, logo remoção desnecessária
diamonds_test[diamonds_test['y'] == 0]

,carat,cut,color,clarity,depth,table,price,x,y,z


In [7]:
# completando as informações perdidas quando o z = 0
diamonds_test.loc[diamonds_test['z']==0 ,'z'] = (diamonds_test['depth']*(diamonds_test['x']+diamonds_test['y']))/2

In [8]:
# buscando valores faltando em outras colunas
diamonds_test[diamonds_test['price'] == 0]

,carat,cut,color,clarity,depth,table,price,x,y,z


In [9]:
diamonds_test[diamonds_test['table'] == 0]

,carat,cut,color,clarity,depth,table,price,x,y,z


In [10]:
diamonds_test['clarity'].unique()

array(['SI1', 'VS1', 'VS2', 'SI2', 'VVS2', 'VVS1', 'I1', 'IF'],
      dtype=object)

In [11]:
diamonds_test['color'].unique()

array(['E', 'I', 'J', 'H', 'F', 'G', 'D'], dtype=object)

In [12]:
diamonds_test['cut'].unique()

array(['Premium', 'Good', 'Very Good', 'Ideal', 'Fair'], dtype=object)

# Exploratory Analysis

In [13]:
# identificando as correlações
diamonds_test.corr()['price']
#outras explorações foram feitas no tableau

carat    0.922207
depth   -0.012077
table    0.127854
price    1.000000
x        0.887904
y        0.866568
z        0.112812
Name: price, dtype: float64

# Data transformation

In [14]:
diamonds_test.loc[diamonds_test['cut'] == 'Fair', 'cut'] = 1
diamonds_test.loc[diamonds_test['cut'] == 'Good', 'cut'] = 2
diamonds_test.loc[diamonds_test['cut'] == 'Very Good', 'cut'] = 3
diamonds_test.loc[diamonds_test['cut'] == 'Premium', 'cut'] = 4
diamonds_test.loc[diamonds_test['cut'] == 'Ideal', 'cut'] = 5

diamonds_rick.loc[diamonds_rick['cut'] == 'Fair', 'cut'] = 1
diamonds_rick.loc[diamonds_rick['cut'] == 'Good', 'cut'] = 2
diamonds_rick.loc[diamonds_rick['cut'] == 'Very Good', 'cut'] = 3
diamonds_rick.loc[diamonds_rick['cut'] == 'Premium', 'cut'] = 4
diamonds_rick.loc[diamonds_rick['cut'] == 'Ideal', 'cut'] = 5

#transformação dos dados para a melhor previsão do modelo
diamonds_test[['carat', 'cut','depth','x','y','z','table','price']].applymap(lambda x: np.log10(x + 1))
diamonds_rick[['carat', 'cut','depth','x','y','z','table']].applymap(lambda x: np.log10(x + 1))

,carat,cut,depth,x,y,z,table
0,0.281033,0.602060,1.808886,0.851258,0.856124,0.689309,1.763428
1,0.139879,0.477121,1.787460,0.754348,0.756636,0.583199,1.792392
2,0.181844,0.778151,1.797960,0.791691,0.789581,0.623249,1.755875
3,0.322219,0.778151,1.794488,0.883661,0.885926,0.706718,1.755875
4,0.515874,0.778151,1.796574,0.974972,0.971740,0.790988,1.763428
...,...,...,...,...,...,...,...
4995,0.113943,0.602060,1.800717,0.720159,0.724276,0.563481,1.778151
4996,0.178977,0.778151,1.798651,0.784617,0.788168,0.619093,1.755875
4997,0.303196,0.602060,1.789581,0.867467,0.870989,0.688420,1.785330
4998,0.117271,0.778151,1.800029,0.728354,0.730782,0.569374,1.755875


# Modeling

In [15]:
#Criando a Coluna price_predicted no dataframe
diamonds_rick['price_predicted'] = 0 

In [16]:
#Maks para as modelagens

#Maks de Clarity
# df de testes
teste_if = diamonds_test['clarity'] == 'IF'
teste_vvs1 = diamonds_test['clarity'] == 'VVS1'
teste_vvs2 = diamonds_test['clarity'] == 'VVS2'
teste_vs1 = diamonds_test['clarity'] == 'VS1'
teste_vs2 = diamonds_test['clarity'] == 'VS2'
teste_si1 = diamonds_test['clarity'] == 'SI1'
teste_si2 = diamonds_test['clarity'] == 'SI2'
teste_i1 = diamonds_test['clarity'] == 'I1'

# df do rick
rick_if = diamonds_rick['clarity'] == 'IF'
rick_vvs1 = diamonds_rick['clarity'] == 'VVS1'
rick_vvs2 = diamonds_rick['clarity'] == 'VVS2'
rick_vs1 = diamonds_rick['clarity'] == 'VS1'
rick_vs2 = diamonds_rick['clarity'] == 'VS2'
rick_si1 = diamonds_rick['clarity'] == 'SI1'
rick_si2 = diamonds_rick['clarity'] == 'SI2'
rick_i1 = diamonds_rick['clarity'] == 'I1'

#Masks de Color
#df de testes
teste_d = diamonds_test['color'] == 'D'
teste_e = diamonds_test['color'] == 'E'
teste_f = diamonds_test['color'] == 'F'
teste_g = diamonds_test['color'] == 'G'
teste_h = diamonds_test['color'] == 'H'
teste_i = diamonds_test['color'] == 'I'
teste_j = diamonds_test['color'] == 'J'
#df do rick
rick_d = diamonds_rick['color'] == 'D'
rick_e = diamonds_rick['color'] == 'E'
rick_f = diamonds_rick['color'] == 'F'
rick_g = diamonds_rick['color'] == 'G'
rick_h = diamonds_rick['color'] == 'H'
rick_i = diamonds_rick['color'] == 'I'
rick_j = diamonds_rick['color'] == 'J'

In [17]:
#listas clarity mask
teste_clarity_list = [
    teste_if, teste_vvs1, teste_vvs2, teste_vs1, teste_vs2, teste_si1,
    teste_si2, teste_i1
]

rick_clarity_list = [
    rick_if, rick_vvs1, rick_vvs2, rick_vs1, rick_vs2, rick_si1, rick_si2,
    rick_i1
]
#listas color mask
teste_color_list = [
    teste_d, teste_e, teste_f, teste_g, teste_h, teste_i, teste_j
]

rick_color_list = [rick_d, rick_e, rick_f, rick_g, rick_h, rick_i, rick_j]

In [18]:
#Loop para criar todas as combinações de modelos 
for clarity in list(zip(teste_clarity_list, rick_clarity_list)):
    for color in list(zip(teste_color_list, rick_color_list)):
        diamonds_rick.loc[clarity[1] & color[1],
                          'price_predicted'] = cria_modelos(
                              diamonds_test[clarity[0] & color[0]],
                              diamonds_rick[clarity[1] & color[1]],
                              ['carat', 'cut','depth','x','y','z','table'], 'price')

# To CSV

In [19]:
#transformando os dados para seus valores originais 
diamonds_test[['carat', 'cut','depth','x','y','z','table','price']].applymap(lambda x: round(np.exp(x),ndigits=2))
diamonds_rick[['carat', 'cut','depth','x','y','z','table', 'price_predicted']].applymap(lambda x: round(np.exp(x),ndigits=2))

diamonds_rick.to_csv('testes_price/price_pred', index=False)
diamonds_rick

C:\Users\guilh\AppData\Local\Temp\ipykernel_13788\3685250210.py:1: RuntimeWarning: overflow encountered in exp
  diamonds_test[['carat', 'cut','depth','x','y','z','table','price']].applymap(lambda x: round(np.exp(x),ndigits=2))
C:\Users\guilh\AppData\Local\Temp\ipykernel_13788\3685250210.py:1: RuntimeWarning: overflow encountered in multiply
  diamonds_test[['carat', 'cut','depth','x','y','z','table','price']].applymap(lambda x: round(np.exp(x),ndigits=2))
C:\Users\guilh\AppData\Local\Temp\ipykernel_13788\3685250210.py:2: RuntimeWarning: overflow encountered in exp
  diamonds_rick[['carat', 'cut','depth','x','y','z','table', 'price_predicted']].applymap(lambda x: round(np.exp(x),ndigits=2))
C:\Users\guilh\AppData\Local\Temp\ipykernel_13788\3685250210.py:2: RuntimeWarning: overflow encountered in multiply
  diamonds_rick[['carat', 'cut','depth','x','y','z','table', 'price_predicted']].applymap(lambda x: round(np.exp(x),ndigits=2))


,carat,cut,color,clarity,depth,table,x,y,z,price_predicted
0,0.91,3,H,SI2,63.4,57.0,6.10,6.18,3.89,3297.009344
1,0.38,2,F,VS2,60.3,61.0,4.68,4.71,2.83,808.839912
2,0.52,5,H,VS2,61.8,56.0,5.19,5.16,3.20,1436.205351
3,1.10,5,G,VS1,61.3,56.0,6.65,6.69,4.09,7445.217268
4,2.28,5,G,SI2,61.6,57.0,8.44,8.37,5.18,14394.037478
...,...,...,...,...,...,...,...,...,...,...
4995,0.30,3,D,SI2,62.2,59.0,4.25,4.30,2.66,523.832810
4996,0.51,5,F,VVS2,61.9,56.0,5.09,5.14,3.16,2131.371771
4997,1.01,3,F,SI1,60.6,60.0,6.37,6.43,3.88,4993.143728
4998,0.31,5,F,VVS1,62.1,56.0,4.35,4.38,2.71,911.662008


# Results


In [20]:
# erro usando a média foi 3980.71
# erro usando a mediana foi 4255.53
# erro usando a primeira modelagem foi 1605.15
# erro após a limpeza dos dados foi  1605.13
# erro usando segundo modelo foi 1578.53
# erro usando mais modelos foi 1254.41
# erro usando modelos com Clarity e Color foi 1113.51
# erro adicionando cut no X foi 1102.37 
# erro usando varios modelos foi 984.97
# erro usando 'carat', 'cut','depth','x','y','z','table' como X foi 785.87
# erro usando GradientBoostingRegressor foi 579.42
# erro transformando os dados 578.12